# Visualization of the Data
### Preparing the data

In [1]:
import pandas as pd
import numpy as np
import glob

def load_preprocess_data():
    path = r'./Data-out' 
    all_files = glob.glob(path + "/hda_dataset_uniform_random_[0-7].csv")
    all_files.sort()
    
    #Load all data
    df = (pd.read_csv(f) for f in all_files)
    
    #Merge all the files with each other
    #Drop all duplicates
    concat_df = pd.concat(df).drop_duplicates().reset_index(drop=True)
    concat_df = concat_df.sort_values(by=['case', 'ts'])
    #reset index
    concat_df = concat_df.reset_index(drop=True)
    #Create a csv file
    #concat_df.to_csv('test1.csv')
    return concat_df

In [22]:
data = load_preprocess_data()

## Visualization 1.0
#### Building a filter for the Data

In [28]:
import plotly_express as px
import plotly.graph_objs as go
import ipywidgets as widgets
from ipywidgets import interactive, HBox, VBox, Layout
import time

data_array = data.event.unique()

trace = px.scatter(data, x="ts", y="case", color="event")
fig = go.FigureWidget(trace)

#Funktion für ComboBox
combobox_arr = []
def update_event_cb(select_event):
    if select_event:
        if select_event not in combobox_arr:
            combobox_arr.append(select_event)
            text_area.value = ', '.join(combobox_arr)
    time.sleep(0.5)
    combobox.value =''
          
#Funktion für Filter zurücksetzen  
def clear_filter(b):
    empty_list =[]
    global combobox_arr
    combobox_arr = empty_list
    selected_data = data.loc[data.event.isin(combobox_arr)]
    trace1 = px.scatter(selected_data, x="ts", y="case", color="event")
    fig.data = []
    fig.update(data = [d.to_plotly_json() for d in trace1.data])
    text_area.value = ''

#Funktion für alle Events im Filter setzen    
def set_filter(b):
    trace1 = px.scatter(data, x="ts", y="case", color="event")
    fig.data = []
    fig.update(data = [d.to_plotly_json() for d in trace1.data])
    text_area.value = ', '.join(data_array)
    
#Funktion für submit button    
def submit_filter(b):
    selected_data = data.loc[data.event.isin(combobox_arr)]
    trace1 = px.scatter(selected_data, x="ts", y="case", color="event")
    fig.data = []
    fig.update(data = [d.to_plotly_json() for d in trace1.data])
    

event_tuple = tuple(data_array)  
#Definition der Widgets
select_multiple = widgets.SelectMultiple(options=data_array, value=event_tuple ,description='Event')
combobox = widgets.Combobox(placeholder="Filter event", options=event_tuple, description="Filter", 
                            ensure_option=True, disabled=False,
                            layout=Layout(width='800px'))

#Definition von Buttons
btn_clearfilter = widgets.Button(description='Clear Filter', button_style='danger')
btn_setfilter = widgets.Button(description='Set All', button_style='info' )
btn_submit = widgets.Button(description='Submit')
text_area = widgets.Textarea(placeholder='Selected Filter', disabled = True, 
                             layout=Layout(width='800px', height='35px'))

#Interaktion der Widgets
cb = interactive(update_event_cb, select_event=combobox)
btn_clearfilter.on_click(clear_filter)
btn_setfilter.on_click(set_filter)
btn_submit.on_click(submit_filter)


VBox([cb, text_area,HBox([btn_submit, btn_setfilter, btn_clearfilter]), fig])

## Visualization 2.0
#### Building Filter with a server


In [23]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

In [39]:
from IPython import display
import os
def show_app(app, port = 9999, width = 700, height = 350, offline = False, in_binder = None):
    url = 'http://localhost:%d' % port    
    iframe = '<a href="{url}" target="_new">Open in new window</a>'.format(url = url)
    
    display.display_html(iframe, raw = True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
        
    return app.run_server(debug=False, host = 'localhost', port=port)

In [25]:
event_arr= data.event.unique()

#append a stylesheet to the html-site
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app_simple = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app_simple.layout = html.Div(children=[
        html.H1(children='Visualization'),
        html.Label(
            [
                "Filter:",
                dcc.Dropdown(id="multi-dynamic-dropdown", options=[
                    {'label': i, 'value': i} for i in event_arr
                ], multi=True, placeholder='Filter nach events...'),
            ]
        ),
        html.Button('Submit', id='submit-btn', style={"background": "grey", "color":"white", "margin-top":"10px"}),
        html.Button('Set All', id='set-all-btn', style={"background":"blue", "color":"white", "margin-top":"10px", "margin-left":"5px"}),
        html.Button('Delete All', id='delete-all-btn', style={"background":"red", "color":"white", "margin-top":"10px", "margin-left":"5px"}),
        dcc.Loading(id="loading-icon", children=[html.Div([
            dcc.Graph(id="graph", style={"width": "100%", "height": "70vh", "display": "inline-block"},
            figure=px.scatter(data, x="ts", y="case", color="event"))
        ])] )
    ]
)

@app_simple.callback(
    Output('graph', 'figure'),
    [Input('submit-btn', 'n_clicks'), Input('set-all-btn', 'n_clicks'), Input('delete-all-btn', 'n_clicks')],
    [State('multi-dynamic-dropdown', 'value')])
def update_output(submit, set_all, delete_all,value):
    ctx = dash.callback_context
    if value == None:
        raise PreventUpdate()
    else:
        selected_data = data.loc[data.event.isin(value)]    
    if (submit != None or set_all != None or delete_all != None):
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == 'submit-btn':           
            return px.scatter(selected_data, x="ts", y="case", color="event")
        elif button_id == 'set-all-btn':
            return px.scatter(data, x="ts", y="case", color="event")
        elif button_id == 'delete-all-btn':
            arr = []
            selected_data = data.loc[data.event.isin(arr)]
            return px.scatter(selected_data, x="ts", y="case", color="event")
        

In [40]:
show_app(app_simple)

Open in new window

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Jan/2020 20:18:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:18:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:18:58] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:18:59] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [25/Jan/2020 20:19:13] "POST /_dash-update-component HTTP/1.1" 200 -
